# Identity Management in LUSID
In this part of the course, we will describe the processes of:
- Setting up personal accounts in LUSID.
- Setting up service accounts in LUSID.
- Getting and using your access tokens.

## Setting up personal accounts in LUSID

We provide the FINBOURNE identity SDKs which can be used to set up personal accounts in LUSID.

In [1]:
from lusid.utilities import ApiClientFactory
import lusid as lu
import pandas as pd
from pprint import pprint
import json

api_factory = ApiClientFactory()

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.10432.0,0.5.3123,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [2]:
import finbourne_identity as identity
from finbourne_identity.rest import ApiException

configuration = identity.Configuration(
    host = f'{api_factory.api_client.configuration.host[:-4]}/identity'
)
configuration.access_token = api_factory.api_client.configuration.access_token

api_client = identity.ApiClient(configuration)

In [3]:
# don't show exception if error is due to upsert
def exception_guard(e, code):
    return e.status and e.status != '400 Bad Request' and e.body and json.loads(e.body)["code"] == code

In [4]:
api_instance = identity.UsersApi(api_client)
create_user_request = {"firstName":"Joe","lastName":"Bloggs","emailAddress":"joe.bloggs@myco.com","login":"joe.bloggs@myco.com","type":"Personal"} # CreateUserRequest | Details of the User to be created
wait_for_reindex = True # bool | Should the request wait until the newly created User is indexed (available in List) before returning (optional) (default to False)
try:
    api_response = api_instance.create_user(create_user_request, wait_for_reindex=wait_for_reindex)
    pprint(api_response)
except ApiException as e:
    if not exception_guard(e, 658):
        print("Exception when calling UsersApi->create_user: %s\n" % e)

Here we create a user called "Joe Bloggs", with their own email and login.
They'll be sent an email to activate their account and to set up login credentials.

## Setting up service accounts in LUSID.

We can also use the FINBOURNE identity SDKs to set up service accounts, for your services to use to authenticate to make LUSID API calls.
To call the LUSID API, a service user must supply a valid username, password, client ID and client secret in order to obtain an API access token, which enables API calls for one hour. After this time, a new token is required.

In [5]:
create_user_request = {"firstName":"service_account@myco.com","lastName":"ServiceAccount","emailAddress":"service_account@myco.com","secondEmailAddress":"service_account_backup@myco.com","login":"service_account@myco.com","type":"Service"} # CreateUserRequest | Details of the User to be created
wait_for_reindex = True # bool | Should the request wait until the newly created User is indexed (available in List) before returning (optional) (default to False)
try:
    api_response = api_instance.create_user(create_user_request, wait_for_reindex=wait_for_reindex)
    pprint(api_response)
except ApiException as e:
    if not exception_guard(e, 658):
        print("Exception when calling UsersApi->create_user: %s\n" % e)

Here we've created a service account, with a backup email address.

## Getting and using your access tokens.

The recommended way to get an access token to query LUSID's APIs is to create an application in LUSID, and use the access token of the application.

Let's create an application and get its access token.

In [6]:
api_instance = identity.ApplicationsApi(api_client)
create_application_request = {"displayName":"IAM example application","clientId":"IAM-example","type":"Native"} # CreateApplicationRequest | Details of the application to be created (optional)

try:
    # [EARLY ACCESS] CreateApplication: Create Application
    api_response = api_instance.create_application(create_application_request=create_application_request)
    pprint(api_response)
except ApiException as e:
    if not exception_guard(e, 157):
        print("Exception when calling ApplicationsApi->create_application: %s\n" % e)

We can then use your username, password, client_id and secret to obtain a short-lived API access token from Okta. The LUSID SDKs can be provided with these as parameters in the api clients, in a secrets JSON file, or as environment variables.

### Personal Access Tokens

Every call made to the LUSID API must be authorised by an API access token. 

The recommended and most secure option is to obtain a short-lived OAuth2.0-compliant token from LUSID’s identity provider, Okta, on demand. 

There may be scenarios where this is not possible, for example if your system does not support Open ID Connect authentication flows. You can create a long-lived personal access token within LUSID and use this to call the API instead. 

We'll now create a Personal Access Token.

In [7]:
from datetime import datetime
from datetime import timedelta
tomorrow = datetime.today() + timedelta(days = 1)
tomorrow = f'{tomorrow.isoformat()}+00:00'

In [10]:
api_instance = identity.PersonalAuthenticationTokensApi(api_client)
create_api_key = {"displayName":"Example key","deactivationDate":tomorrow} # CreateApiKey | The request to create a new Personal Access Token

try:
    # [EARLY ACCESS] CreateApiKey: Create a Personal Access Token
    api_response = api_instance.create_api_key(create_api_key)
    pprint(api_response)
except ApiException as e:
    if not exception_guard(e, 659):
        print("Exception when calling PersonalAuthenticationTokensApi->create_api_key: %s\n" % e)